# Local inference on Docker Image

- Tensorflow Serving Model:
    - https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html#tensorflow-serving-model


In [76]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [77]:
import os
%store -r training_job_name

no stored variable or alias training_job_name


In [78]:
training_job_name = 'bert2tweet-2020-07-08-07-58-27-895'
print(training_job_name)

bert2tweet-2020-07-08-07-58-27-895


In [79]:
container_image = '057716757052.dkr.ecr.us-west-2.amazonaws.com/sagemaker-tensorflow-serving:2.0.0-cpu'


In [80]:
from sagemaker.tensorflow.serving import Model

model = Model(model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name),
              role=role,
              framework_version='2.0.0',
              entry_point='inference.py',
              image=container_image
             ) # Elastic Inference does not yet support TF 2.1.0 as of sagemaker==1.56.1

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


- When an error occurs, run "docker container ls" on terminal and if exists and stop with "docker container rm <Container ID>

In [81]:
%%time

instance_type='local'
local_endpoint = model.deploy(initial_instance_count = 1,
                             instance_type = instance_type,
                             wait=False)


Attaching to tmp159pek3h_algo-1-4b38r_1
algo-1-4b38r_1  | INFO:__main__:starting services
algo-1-4b38r_1  | INFO:__main__:nginx config: 
algo-1-4b38r_1  | load_module modules/ngx_http_js_module.so;
algo-1-4b38r_1  | 
algo-1-4b38r_1  | worker_processes auto;
algo-1-4b38r_1  | daemon off;
algo-1-4b38r_1  | pid /tmp/nginx.pid;
algo-1-4b38r_1  | error_log  /dev/stderr error;
algo-1-4b38r_1  | 
algo-1-4b38r_1  | worker_rlimit_nofile 4096;
algo-1-4b38r_1  | 
algo-1-4b38r_1  | events {
algo-1-4b38r_1  |   worker_connections 2048;
algo-1-4b38r_1  | }
algo-1-4b38r_1  | 
algo-1-4b38r_1  | http {
algo-1-4b38r_1  |   include /etc/nginx/mime.types;
algo-1-4b38r_1  |   default_type application/json;
algo-1-4b38r_1  |   access_log /dev/stdout combined;
algo-1-4b38r_1  |   js_include tensorflow-serving.js;
algo-1-4b38r_1  | 
algo-1-4b38r_1  |   upstream tfs_upstream {
algo-1-4b38r_1  |     server localhost:8501;
algo-1-4b38r_1  |   }
algo-1-4b38r_1  | 
algo-1-4b38r_1  |   upstream gunicorn_upstream {


## Invoke the endpoint without custom Docker image

In [62]:
local_endpoint.endpoint

'sagemaker-tensorflow-serving-2020-07-09-04-50-37-884'

In [84]:
reviews = ["This is great", "The weather is wonderful"]
predicted_classes = local_endpoint.predict(reviews)
predicted_classes

algo-1-4b38r_1  | INFO:python_service:http://gunicorn_upstream/invocations
algo-1-4b38r_1  | INFO:tfs_utils:sagemaker tfs attributes: 
algo-1-4b38r_1  | {}
algo-1-4b38r_1  | instances: 
algo-1-4b38r_1  |  ['This is great', 'The weather is wonderful']
algo-1-4b38r_1  | instance:  This is great
algo-1-4b38r_1  | transformed_instance: 
algo-1-4b38r_1  |  {'input_ids': [101, 2023, 2003, 2307, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'input_mask': [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

[[-1.37874043,
  -1.19526446,
  -1.26716495,
  -1.1868751,
  7.1464591,
  -1.53433263,
  -2.15244746,
  -1.74911451,
  -2.27858257,
  -1.96475625],
 [-1.3166939,
  -1.18195844,
  -1.41097331,
  -1.0972333,
  7.164114,
  -1.53150845,
  -2.15987682,
  -1.74124718,
  -2.26538515,
  -1.96443963]]

algo-1-4b38r_1  | 172.18.0.1 - - [09/Jul/2020:07:22:27 +0000] "POST /invocations HTTP/1.1" 200 256 "-" "-"
algo-1-4b38r_1  | 172.18.0.1 - - [09/Jul/2020:07:25:02 +0000] "GET /ping HTTP/1.1" 200 0 "-" "-"
algo-1-4b38r_1  | 172.18.0.1 - - [09/Jul/2020:07:27:41 +0000] "GET /ping HTTP/1.1" 200 0 "-" "-"
